<a href="https://colab.research.google.com/github/beza-lab/KAIM-WEEK-5/blob/main/Task4_Model_Comparison_Selection_Task%205_Model_Interpretability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install transformers datasets seqeval torch evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00


In [18]:
from transformers import XLMRobertaTokenizer, XLMRobertaForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset
from evaluate import load
import torch

from google.colab import files

# upload the file
uploaded = files.upload()

Saving labeled_dataset.conll to labeled_dataset.conll


In [19]:
import pandas as pd

# Get the filename from the uploaded dictionary
filename = list(uploaded.keys())[0]
print(f'Loading file: {filename}')

# Initialize lists to hold tokens and labels
tokens = []
labels = []

# Read the file line by line
with open(filename, 'r') as file:
    for line in file:
        line = line.strip()
        if line:  # Skip empty lines
            parts = line.split(maxsplit=1)  # Split by the first space only
            if len(parts) == 2:  # Ensure there are exactly 2 parts
                token, label = parts
                tokens.append(token)
                labels.append(label)
            else:
                print(f"Skipping line: {line}")  # Debugging output for lines that don't match the format

# Create a DataFrame
df = pd.DataFrame({'Token': tokens, 'Label': labels})


Loading file: labeled_dataset.conll


In [20]:
# Create a label mapping
label_mapping = {
    "O": 0,
    "I-LOC": 1,
    "I-PRICE": 2,
    "I-Product": 3,
    # Add other labels as needed
}

# Map the labels in the DataFrame
df['Label'] = df['Label'].map(label_mapping)

# Drop any rows with NaN values after mapping
df.dropna(inplace=True)

# Convert labels to integer type
df['Label'] = df['Label'].astype(int)

# Display the DataFrame after mapping
print(df.head(5))

    Token  Label
0   እነዚህን      0
2      እና      0
3      ላይ      0
4  ማስታወቂያ      0
5   አይታችሁ      0


In [21]:
from transformers import XLMRobertaTokenizerFast, XLMRobertaForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset
from evaluate import load
import torch

# Load the fast tokenizer and model for XLM-Roberta
xlm_roberta_tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")
xlm_roberta_model = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(label_mapping))

def tokenize_and_align_labels_xlm_roberta(examples):
    tokenized_inputs = xlm_roberta_tokenizer(
        examples['Token'].tolist(),
        padding=True,
        truncation=True,
        is_split_into_words=True,
    )

    labels = []
    word_ids = tokenized_inputs.word_ids(batch_index=0)  # Assuming batch_size is 1
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            label_ids.append(examples['Label'].values[word_idx])
        else:
            label_ids.append(-100)
        previous_word_idx = word_idx
    tokenized_inputs["labels"] = label_ids
    return tokenized_inputs

# Apply the function to each sentence
tokenized_datasets_xlm_roberta = [tokenize_and_align_labels_xlm_roberta(sentence) for _, sentence in df.groupby(df.index // 10)]

# Convert lists to PyTorch tensors
input_ids_xlm_roberta = [torch.tensor(item['input_ids'], dtype=torch.long) for item in tokenized_datasets_xlm_roberta]
attention_mask_xlm_roberta = [torch.tensor(item['attention_mask'], dtype=torch.long) for item in tokenized_datasets_xlm_roberta]
labels_xlm_roberta = [torch.tensor(item['labels'], dtype=torch.long) for item in tokenized_datasets_xlm_roberta]

# Pad sequences to ensure they have the same length
input_ids_xlm_roberta = torch.nn.utils.rnn.pad_sequence(input_ids_xlm_roberta, batch_first=True, padding_value=xlm_roberta_tokenizer.pad_token_id)
attention_mask_xlm_roberta = torch.nn.utils.rnn.pad_sequence(attention_mask_xlm_roberta, batch_first=True, padding_value=0)
labels_xlm_roberta = torch.nn.utils.rnn.pad_sequence(labels_xlm_roberta, batch_first=True, padding_value=-100)

# Combine into a dataset
dataset_xlm_roberta = Dataset.from_dict({
    'input_ids': input_ids_xlm_roberta,
    'attention_mask': attention_mask_xlm_roberta,
    'labels': labels_xlm_roberta
})

# Set up training arguments for XLM-Roberta
training_args_xlm_roberta = TrainingArguments(
    output_dir='./results_xlm_roberta',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Fine-tune the model
trainer_xlm_roberta = Trainer(
    model=xlm_roberta_model,
    args=training_args_xlm_roberta,
    train_dataset=dataset_xlm_roberta,
    eval_dataset=dataset_xlm_roberta,
)

trainer_xlm_roberta.train()

# Evaluate the model
results_xlm_roberta = trainer_xlm_roberta.evaluate()
print(results_xlm_roberta)

# Save the model
xlm_roberta_model.save_pretrained('./fine-tuned-xlm-roberta-model')
xlm_roberta_tokenizer.save_pretrained('./fine-tuned-xlm-roberta-model')


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

...

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,No log,0.012252
2,No log,0.002300
3,No log,0.000789


{'eval_loss': 0.0007893109577707946, 'eval_runtime': 180.7926, 'eval_samples_per_second': 7.085, 'eval_steps_per_second': 0.448, 'epoch': 3.0}


('./fine-tuned-xlm-roberta-model/tokenizer_config.json',
 './fine-tuned-xlm-roberta-model/special_tokens_map.json',
 './fine-tuned-xlm-roberta-model/sentencepiece.bpe.model',
 './fine-tuned-xlm-roberta-model/added_tokens.json',
 './fine-tuned-xlm-roberta-model/tokenizer.json')